In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
plt.rcParams.update({'figure.max_open_warning': 0})
import matplotlib
import glob
import os

from scipy.stats import trim_mean
from statsmodels import robust
from matplotlib import font_manager, rc
%matplotlib inline
from matplotlib import cm 
import gc  # garbage collection
from sklearn.cluster import KMeans  # k-means 군집분석
from sklearn.metrics import silhouette_score  # 군집분석 평가를 위한 silhouette 값 계산

pd.set_option('max_columns', 20, 'max_rows', 20)

%matplotlib inline

# 한글 폰트 설정|
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

import datetime

In [2]:
hist = pd.read_csv('0930피쳐수정.csv', encoding='cp949')

train = pd.read_csv('../data/stk_hld_train.csv')
test = pd.read_csv('../data/stk_hld_test.csv')

In [3]:
hist = hist[['계좌번호', '기준일자', '종목코드', '주당액면가', '종목보유잔고수량', '종목보유잔고수량_shift',
       '전체종목평가금액', '전체종목평가금액_shift', '일_매수매도_수량', '계좌별_종목별_총보유기간_real']]

In [4]:
hist

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,계좌별_종목별_총보유기간_real
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,351
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,351
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,351
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,351
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,351
...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,62
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,62
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,62
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,4


In [5]:
hist['해당일시_주당금액'] = hist.전체종목평가금액 / hist.종목보유잔고수량

# 1. 종목별 피쳐

## 1-1. 매수 피쳐

In [6]:
hist_mesu = hist.query('일_매수매도_수량 > 0')

In [7]:
#1. 계좌별 매수 수량 평균
base = hist_mesu.groupby(['계좌번호'])['일_매수매도_수량'].mean().reset_index()
base = base.rename({'계좌번호': 'act_id', '일_매수매도_수량' : '계좌별_평균_일매수_수량'}, axis=1)

In [8]:
#2. 계좌별 매수 최고 매수 수량
temp = hist_mesu.groupby(['계좌번호'])['일_매수매도_수량'].max().reset_index()
temp = temp.rename({'계좌번호': 'act_id', '일_매수매도_수량' : '계좌별_최고_일매수_수량'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [9]:
#3. 계좌별 매수 최소 주당금액 
temp = hist_mesu.groupby(['계좌번호'])['해당일시_주당금액'].min().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수_최소_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [10]:
#4. 계좌별 매수 최대 주당금액
temp = hist_mesu.groupby(['계좌번호'])['해당일시_주당금액'].max().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수_최대_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [11]:
#5. 계좌별 매수 평균 주당금액
temp = hist_mesu.groupby(['계좌번호'])['해당일시_주당금액'].mean().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수_평균_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left');base

,act_id,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,22.287356,200.0,1645.0,667000.0,58984.597701
1,0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa...,279.235294,5000.0,507.0,286500.0,67341.573529
2,0008d1bfac569bdb998897e2eb8519dec06b9277a686f4...,451.188889,8388.0,706.0,127500.0,27061.772222
3,000a83a961cbcbce40673aed998effbd462fb10dcac8e6...,1.380952,4.0,4875.0,377000.0,94089.285714
4,000b92ca83a96312a0909b1f489340f4d399c86555012f...,44.911111,727.0,10250.0,360000.0,55766.333333
...,...,...,...,...,...,...
9987,ffea89aa464a4fcf8a5416989d21deef4e748192ba17bf...,2256.506494,20000.0,662.0,71000.0,2527.571429
9988,ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7ba...,978.066667,17694.0,233.0,421000.0,21687.560000
9989,ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae...,209.500000,1300.0,530.0,1224000.0,104058.000000
9990,fff7d563e388744fdc1ca90153ec0161d1345511cb9070...,2215.342105,17324.0,277.0,156500.0,22415.605263


## 1-2. 계좌별 이전 주식 보유기간

In [12]:
hist_period = hist.drop_duplicates(['계좌번호', '종목코드']);

In [13]:
#2. 계좌별 최대 보유기간
temp = hist_period.groupby(['계좌번호'])['계좌별_종목별_총보유기간_real'].max().reset_index()
temp = temp.rename({'계좌번호': 'act_id', '계좌별_종목별_총보유기간_real' : '계좌별_최대보유기간'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

#1. 계좌별 평균 보유기간
temp = hist_period.groupby(['계좌번호'])['계좌별_종목별_총보유기간_real'].mean().reset_index()
temp = temp.rename({'계좌번호': 'act_id', '계좌별_종목별_총보유기간_real' : '계좌별_평균보유기간'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left');base

,act_id,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액,계좌별_최대보유기간,계좌별_평균보유기간
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,22.287356,200.0,1645.0,667000.0,58984.597701,351,91.851852
1,0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa...,279.235294,5000.0,507.0,286500.0,67341.573529,145,20.000000
2,0008d1bfac569bdb998897e2eb8519dec06b9277a686f4...,451.188889,8388.0,706.0,127500.0,27061.772222,256,58.294118
3,000a83a961cbcbce40673aed998effbd462fb10dcac8e6...,1.380952,4.0,4875.0,377000.0,94089.285714,436,79.500000
4,000b92ca83a96312a0909b1f489340f4d399c86555012f...,44.911111,727.0,10250.0,360000.0,55766.333333,588,108.833333
...,...,...,...,...,...,...,...,...
9987,ffea89aa464a4fcf8a5416989d21deef4e748192ba17bf...,2256.506494,20000.0,662.0,71000.0,2527.571429,1166,234.000000
9988,ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7ba...,978.066667,17694.0,233.0,421000.0,21687.560000,318,40.073171
9989,ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae...,209.500000,1300.0,530.0,1224000.0,104058.000000,448,91.407407
9990,fff7d563e388744fdc1ca90153ec0161d1345511cb9070...,2215.342105,17324.0,277.0,156500.0,22415.605263,70,12.172414


## 1-3. 계좌별 매수,매도 합친거

In [14]:
hist

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,계좌별_종목별_총보유기간_real,해당일시_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,351,667000.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,351,635000.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,351,605000.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,351,526000.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,351,370500.0
...,...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,62,49150.0
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,62,49100.0
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,62,NaN
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,4,71500.0


In [15]:
#1. 계좌별 매수매도 수량 평균
temp = hist.groupby(['계좌번호'])['일_매수매도_수량'].mean().reset_index()
temp = temp.rename({'계좌번호': 'act_id', '일_매수매도_수량' : '계좌별_평균_일매수매도_수량'}, axis=1)
base = pd.merge(base, temp, on='act_id', how='left')

In [16]:
#2. 계좌별 매수매도 최고 매수 수량
temp = hist.groupby(['계좌번호'])['일_매수매도_수량'].max().reset_index()
temp = temp.rename({'계좌번호': 'act_id', '일_매수매도_수량' : '계좌별_최고_일매수매도_수량'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [17]:
#3. 계좌별 매수매도 최소 주당금액 
temp = hist.groupby(['계좌번호'])['해당일시_주당금액'].min().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수매도_최소_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [18]:
#4. 계좌별 매수매도 최대 주당금액
temp = hist.groupby(['계좌번호'])['해당일시_주당금액'].max().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수매도_최대_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left')

In [19]:
#5. 계좌별 매수매도 평균 주당금액
temp = hist.groupby(['계좌번호'])['해당일시_주당금액'].mean().reset_index();temp
temp = temp.rename({'계좌번호': 'act_id', '해당일시_주당금액' : '계좌별_매수매도_평균_주당금액'}, axis=1);temp
base = pd.merge(base, temp, on='act_id', how='left');base

,act_id,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액,계좌별_최대보유기간,계좌별_평균보유기간,계좌별_평균_일매수매도_수량,계좌별_최고_일매수매도_수량,계좌별_매수매도_최소_주당금액,계좌별_매수매도_최대_주당금액,계좌별_매수매도_평균_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,22.287356,200.0,1645.0,667000.0,58984.597701,351,91.851852,3.174603,200.0,1645.0,667000.0,55868.009709
1,0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa...,279.235294,5000.0,507.0,286500.0,67341.573529,145,20.000000,50.590909,5000.0,507.0,286500.0,64878.679487
2,0008d1bfac569bdb998897e2eb8519dec06b9277a686f4...,451.188889,8388.0,706.0,127500.0,27061.772222,256,58.294118,28.109215,8388.0,706.0,127500.0,25831.962343
3,000a83a961cbcbce40673aed998effbd462fb10dcac8e6...,1.380952,4.0,4875.0,377000.0,94089.285714,436,79.500000,1.041667,4.0,4875.0,377000.0,94089.285714
4,000b92ca83a96312a0909b1f489340f4d399c86555012f...,44.911111,727.0,10250.0,360000.0,55766.333333,588,108.833333,39.653061,727.0,10250.0,360000.0,55557.282609
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,ffea89aa464a4fcf8a5416989d21deef4e748192ba17bf...,2256.506494,20000.0,662.0,71000.0,2527.571429,1166,234.000000,2256.506494,20000.0,662.0,71000.0,2527.571429
9988,ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7ba...,978.066667,17694.0,233.0,421000.0,21687.560000,318,40.073171,14.770833,17694.0,233.0,421000.0,20076.483146
9989,ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae...,209.500000,1300.0,530.0,1224000.0,104058.000000,448,91.407407,58.218750,1300.0,530.0,1224000.0,102016.250000
9990,fff7d563e388744fdc1ca90153ec0161d1345511cb9070...,2215.342105,17324.0,277.0,156500.0,22415.605263,70,12.172414,119.485714,17324.0,277.0,156500.0,20903.738095


In [20]:
base

,act_id,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액,계좌별_최대보유기간,계좌별_평균보유기간,계좌별_평균_일매수매도_수량,계좌별_최고_일매수매도_수량,계좌별_매수매도_최소_주당금액,계좌별_매수매도_최대_주당금액,계좌별_매수매도_평균_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,22.287356,200.0,1645.0,667000.0,58984.597701,351,91.851852,3.174603,200.0,1645.0,667000.0,55868.009709
1,0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa...,279.235294,5000.0,507.0,286500.0,67341.573529,145,20.000000,50.590909,5000.0,507.0,286500.0,64878.679487
2,0008d1bfac569bdb998897e2eb8519dec06b9277a686f4...,451.188889,8388.0,706.0,127500.0,27061.772222,256,58.294118,28.109215,8388.0,706.0,127500.0,25831.962343
3,000a83a961cbcbce40673aed998effbd462fb10dcac8e6...,1.380952,4.0,4875.0,377000.0,94089.285714,436,79.500000,1.041667,4.0,4875.0,377000.0,94089.285714
4,000b92ca83a96312a0909b1f489340f4d399c86555012f...,44.911111,727.0,10250.0,360000.0,55766.333333,588,108.833333,39.653061,727.0,10250.0,360000.0,55557.282609
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,ffea89aa464a4fcf8a5416989d21deef4e748192ba17bf...,2256.506494,20000.0,662.0,71000.0,2527.571429,1166,234.000000,2256.506494,20000.0,662.0,71000.0,2527.571429
9988,ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7ba...,978.066667,17694.0,233.0,421000.0,21687.560000,318,40.073171,14.770833,17694.0,233.0,421000.0,20076.483146
9989,ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae...,209.500000,1300.0,530.0,1224000.0,104058.000000,448,91.407407,58.218750,1300.0,530.0,1224000.0,102016.250000
9990,fff7d563e388744fdc1ca90153ec0161d1345511cb9070...,2215.342105,17324.0,277.0,156500.0,22415.605263,70,12.172414,119.485714,17324.0,277.0,156500.0,20903.738095


In [21]:
base.to_csv('../data/계좌별피쳐.csv', encoding='cp949', index=False)

# 2. 종목별 피쳐

In [22]:
hist

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,계좌별_종목별_총보유기간_real,해당일시_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,351,667000.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,351,635000.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,351,605000.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,351,526000.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,351,370500.0
...,...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,62,49150.0
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,62,49100.0
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,62,NaN
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,4,71500.0


## 2-1. 매수피쳐

In [23]:
hist_mesu = hist.query('일_매수매도_수량 > 0')

In [24]:
#1. 종목별 매수 수량 평균
base_1 = hist_mesu.groupby(['종목코드'])['일_매수매도_수량'].mean().reset_index()
base_1 = base_1.rename({'종목코드': 'iem_cd', '일_매수매도_수량' : '종목별_평균_일매수_수량'}, axis=1)

In [25]:
#2. 종목별 매수 최고 매수 수량
temp = hist_mesu.groupby(['종목코드'])['일_매수매도_수량'].max().reset_index()
temp = temp.rename({'종목코드': 'iem_cd', '일_매수매도_수량' : '종목별_최고_일매수_수량'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [26]:
#3. 종목별 매수 최소 주당금액 
temp = hist_mesu.groupby(['종목코드'])['해당일시_주당금액'].min().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수_최소_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [27]:
#4. 종목별 매수 최대 주당금액
temp = hist_mesu.groupby(['종목코드'])['해당일시_주당금액'].max().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수_최대_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [28]:
#5. 종목별 매수 평균 주당금액
temp = hist_mesu.groupby(['종목코드'])['해당일시_주당금액'].mean().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수_평균_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left');base_1

,iem_cd,종목별_평균_일매수_수량,종목별_최고_일매수_수량,종목별_매수_최소_주당금액,종목별_매수_최대_주당금액,종목별_매수_평균_주당금액
0,A000020,173.485356,15000.0,4985.0,29350.0,18940.873431
1,A000030,293.441520,30243.0,8540.0,19250.0,15511.754386
2,A000040,2540.628664,104570.0,261.0,1430.0,786.771987
3,A000050,170.081633,1700.0,9390.0,15600.0,11490.204082
4,A000060,481.441704,50000.0,9180.0,26000.0,17226.928251
...,...,...,...,...,...,...
2921,A371460,115.192308,1000.0,10035.0,11705.0,11044.615385
2922,A371470,733.315789,7750.0,9810.0,11225.0,10660.789474
2923,A371870,4.500000,5.0,9925.0,10180.0,10052.500000
2924,A372330,283.923077,1500.0,9610.0,10470.0,9958.076923


## 2-2. 종목별 이전 주식 보유기간

In [29]:
hist_period = hist.drop_duplicates(['계좌번호', '종목코드']);hist_period

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,계좌별_종목별_총보유기간_real,해당일시_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,351,667000.0
8,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-27,A005070,1000.0,10.0,0.0,152500.0,0.0,10.0,20,15250.0
10,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-20,A005930,100.0,10.0,0.0,440500.0,0.0,10.0,2,44050.0
12,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-26,A033270,500.0,5.0,0.0,130250.0,0.0,5.0,76,26050.0
16,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-06-11,A036000,500.0,20.0,0.0,99300.0,0.0,20.0,134,4965.0
...,...,...,...,...,...,...,...,...,...,...,...
2573796,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-01-30,A253450,500.0,1.0,0.0,84700.0,0.0,1.0,131,84700.0
2573798,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-17,A264450,500.0,2.0,0.0,52900.0,0.0,2.0,9,26450.0
2573800,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-07,A267320,100.0,15.0,0.0,81600.0,0.0,15.0,48,5440.0
2573802,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-14,A293490,100.0,2.0,0.0,147600.0,0.0,2.0,62,73800.0


In [30]:
#2. 종목별 최대 보유기간
temp = hist_period.groupby(['종목코드'])['계좌별_종목별_총보유기간_real'].max().reset_index()
temp = temp.rename({'종목코드': 'iem_cd', '계좌별_종목별_총보유기간_real' : '종목별_최대보유기간'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

#1. 종목별 평균 보유기간
temp = hist_period.groupby(['종목코드'])['계좌별_종목별_총보유기간_real'].mean().reset_index()
temp = temp.rename({'종목코드': 'iem_cd', '계좌별_종목별_총보유기간_real' : '종목별_평균보유기간'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left');base_1

,iem_cd,종목별_평균_일매수_수량,종목별_최고_일매수_수량,종목별_매수_최소_주당금액,종목별_매수_최대_주당금액,종목별_매수_평균_주당금액,종목별_최대보유기간,종목별_평균보유기간
0,A000020,173.485356,15000.0,4985.0,29350.0,18940.873431,770,36.502825
1,A000030,293.441520,30243.0,8540.0,19250.0,15511.754386,738,58.313253
2,A000040,2540.628664,104570.0,261.0,1430.0,786.771987,481,37.780220
3,A000050,170.081633,1700.0,9390.0,15600.0,11490.204082,165,31.846154
4,A000060,481.441704,50000.0,9180.0,26000.0,17226.928251,733,109.184466
...,...,...,...,...,...,...,...,...
2921,A371460,115.192308,1000.0,10035.0,11705.0,11044.615385,11,4.181818
2922,A371470,733.315789,7750.0,9810.0,11225.0,10660.789474,15,7.200000
2923,A371870,4.500000,5.0,9925.0,10180.0,10052.500000,5,3.000000
2924,A372330,283.923077,1500.0,9610.0,10470.0,9958.076923,7,2.375000


## 2-3. 종목별 매수,매도 합친거

In [31]:
hist

,계좌번호,기준일자,종목코드,주당액면가,종목보유잔고수량,종목보유잔고수량_shift,전체종목평가금액,전체종목평가금액_shift,일_매수매도_수량,계좌별_종목별_총보유기간_real,해당일시_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-21,A002270,5000.0,1.0,0.0,667000.0,0.0,1.0,351,667000.0
1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-03-28,A002270,5000.0,2.0,1.0,1270000.0,667000.0,1.0,351,635000.0
2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-05-07,A002270,5000.0,3.0,2.0,1815000.0,1270000.0,1.0,351,605000.0
3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2019-07-16,A002270,5000.0,4.0,3.0,2104000.0,1815000.0,1.0,351,526000.0
4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,2020-01-29,A002270,5000.0,8.0,4.0,2964000.0,2104000.0,4.0,351,370500.0
...,...,...,...,...,...,...,...,...,...,...,...
2573804,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,100.0,4.0,3.0,196600.0,198900.0,1.0,62,49150.0
2573805,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,100.0,5.0,4.0,245500.0,196600.0,1.0,62,49100.0
2573806,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,100.0,0.0,5.0,0.0,245500.0,-5.0,62,NaN
2573807,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-10,A298040,5000.0,15.0,0.0,1072500.0,0.0,15.0,4,71500.0


In [32]:
#1. 계좌별 매수매도 수량 평균
temp = hist.groupby(['종목코드'])['일_매수매도_수량'].mean().reset_index()
temp = temp.rename({'종목코드': 'iem_cd', '일_매수매도_수량' : '종목별_평균_일매수매도_수량'}, axis=1)
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [33]:
#2. 계좌별 매수매도 최고 매수 수량
temp = hist.groupby(['종목코드'])['일_매수매도_수량'].max().reset_index()
temp = temp.rename({'종목코드': 'iem_cd', '일_매수매도_수량' : '종목별_최고_일매수매도_수량'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [34]:
#3. 계좌별 매수매도 최소 주당금액 
temp = hist.groupby(['종목코드'])['해당일시_주당금액'].min().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수매도_최소_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [35]:
#4. 계좌별 매수매도 최대 주당금액
temp = hist.groupby(['종목코드'])['해당일시_주당금액'].max().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수매도_최대_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left')

In [36]:
#5. 계좌별 매수매도 평균 주당금액
temp = hist.groupby(['종목코드'])['해당일시_주당금액'].mean().reset_index();temp
temp = temp.rename({'종목코드': 'iem_cd', '해당일시_주당금액' : '종목별_매수매도_평균_주당금액'}, axis=1);temp
base_1 = pd.merge(base_1, temp, on='iem_cd', how='left');base

,act_id,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액,계좌별_최대보유기간,계좌별_평균보유기간,계좌별_평균_일매수매도_수량,계좌별_최고_일매수매도_수량,계좌별_매수매도_최소_주당금액,계좌별_매수매도_최대_주당금액,계좌별_매수매도_평균_주당금액
0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,22.287356,200.0,1645.0,667000.0,58984.597701,351,91.851852,3.174603,200.0,1645.0,667000.0,55868.009709
1,0007f3d10282e6f9de1eb445a64eb1f13e9ed2308cfbfa...,279.235294,5000.0,507.0,286500.0,67341.573529,145,20.000000,50.590909,5000.0,507.0,286500.0,64878.679487
2,0008d1bfac569bdb998897e2eb8519dec06b9277a686f4...,451.188889,8388.0,706.0,127500.0,27061.772222,256,58.294118,28.109215,8388.0,706.0,127500.0,25831.962343
3,000a83a961cbcbce40673aed998effbd462fb10dcac8e6...,1.380952,4.0,4875.0,377000.0,94089.285714,436,79.500000,1.041667,4.0,4875.0,377000.0,94089.285714
4,000b92ca83a96312a0909b1f489340f4d399c86555012f...,44.911111,727.0,10250.0,360000.0,55766.333333,588,108.833333,39.653061,727.0,10250.0,360000.0,55557.282609
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9987,ffea89aa464a4fcf8a5416989d21deef4e748192ba17bf...,2256.506494,20000.0,662.0,71000.0,2527.571429,1166,234.000000,2256.506494,20000.0,662.0,71000.0,2527.571429
9988,ffecc8e2823943ef1a7b0d9c6512f716a88032a1bdf7ba...,978.066667,17694.0,233.0,421000.0,21687.560000,318,40.073171,14.770833,17694.0,233.0,421000.0,20076.483146
9989,ffee399294db1c721e010cbf3dae5c8a1f140dbaf2dcae...,209.500000,1300.0,530.0,1224000.0,104058.000000,448,91.407407,58.218750,1300.0,530.0,1224000.0,102016.250000
9990,fff7d563e388744fdc1ca90153ec0161d1345511cb9070...,2215.342105,17324.0,277.0,156500.0,22415.605263,70,12.172414,119.485714,17324.0,277.0,156500.0,20903.738095


In [37]:
base_1

,iem_cd,종목별_평균_일매수_수량,종목별_최고_일매수_수량,종목별_매수_최소_주당금액,종목별_매수_최대_주당금액,종목별_매수_평균_주당금액,종목별_최대보유기간,종목별_평균보유기간,종목별_평균_일매수매도_수량,종목별_최고_일매수매도_수량,종목별_매수매도_최소_주당금액,종목별_매수매도_최대_주당금액,종목별_매수매도_평균_주당금액
0,A000020,173.485356,15000.0,4985.0,29350.0,18940.873431,770,36.502825,12.010585,15000.0,4985.0,30300.0,19024.849398
1,A000030,293.441520,30243.0,8540.0,19250.0,15511.754386,738,58.313253,32.886364,30243.0,8540.0,19250.0,15395.465686
2,A000040,2540.628664,104570.0,261.0,1430.0,786.771987,481,37.780220,51.002070,104570.0,261.0,1430.0,802.842391
3,A000050,170.081633,1700.0,9390.0,15600.0,11490.204082,165,31.846154,0.133333,1700.0,9390.0,15600.0,11311.935484
4,A000060,481.441704,50000.0,9180.0,26000.0,17226.928251,733,109.184466,5.348297,50000.0,9180.0,26000.0,17161.656805
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,A371460,115.192308,1000.0,10035.0,11705.0,11044.615385,11,4.181818,98.500000,1000.0,10035.0,11705.0,11029.137931
2922,A371470,733.315789,7750.0,9810.0,11225.0,10660.789474,15,7.200000,1.576923,7750.0,9810.0,11225.0,10748.958333
2923,A371870,4.500000,5.0,9925.0,10180.0,10052.500000,5,3.000000,1.333333,5.0,9925.0,10180.0,10052.500000
2924,A372330,283.923077,1500.0,9610.0,10470.0,9958.076923,7,2.375000,167.750000,1500.0,9610.0,10470.0,9955.000000


In [38]:
base_1.to_csv('../data/종목별피쳐.csv', encoding='cp949', index=False)

In [39]:
train = train.iloc[:,:2];train

,act_id,iem_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310
...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930


In [40]:
test = test.iloc[:,:2];test

,act_id,iem_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490
...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630


In [41]:
features_final = pd.concat([train, test])

In [42]:
features_final = pd.merge(features_final, base, on = 'act_id', how='left')
features_final = pd.merge(features_final, base_1, on = 'iem_cd', how='left')

In [43]:
features_final

,act_id,iem_cd,계좌별_평균_일매수_수량,계좌별_최고_일매수_수량,계좌별_매수_최소_주당금액,계좌별_매수_최대_주당금액,계좌별_매수_평균_주당금액,계좌별_최대보유기간,계좌별_평균보유기간,계좌별_평균_일매수매도_수량,...,종목별_매수_최소_주당금액,종목별_매수_최대_주당금액,종목별_매수_평균_주당금액,종목별_최대보유기간,종목별_평균보유기간,종목별_평균_일매수매도_수량,종목별_최고_일매수매도_수량,종목별_매수매도_최소_주당금액,종목별_매수매도_최대_주당금액,종목별_매수매도_평균_주당금액
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,391.306122,6000.0,1205.0,2639000.0,77150.068027,434,52.291139,10.407942,...,15050.0,54500.0,33617.870722,1199,86.673759,5.019541,4180.0,15050.0,54700.0,33766.292382
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,391.306122,6000.0,1205.0,2639000.0,77150.068027,434,52.291139,10.407942,...,37450.0,2861000.0,102148.313648,1227,160.183769,4.892984,33663.0,37450.0,2861000.0,106562.678811
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,391.306122,6000.0,1205.0,2639000.0,77150.068027,434,52.291139,10.407942,...,2745.0,28350.0,14512.959705,724,64.793496,3.230341,12624.0,2745.0,28350.0,14610.070841
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,391.306122,6000.0,1205.0,2639000.0,77150.068027,434,52.291139,10.407942,...,3695.0,12800.0,7718.112416,923,63.845113,24.012728,13096.0,3695.0,12800.0,7734.231486
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,391.306122,6000.0,1205.0,2639000.0,77150.068027,434,52.291139,10.407942,...,18700.0,50100.0,33056.417112,640,70.685950,1.714903,1600.0,18700.0,50100.0,33346.953642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752063,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,27.466825,5000.0,890.0,388500.0,66568.856635,1179,348.857143,3.376376,...,129500.0,304000.0,200097.985348,1009,83.784314,3.005714,3000.0,129500.0,304000.0,199921.538462
752064,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,27.466825,5000.0,890.0,388500.0,66568.856635,1179,348.857143,3.376376,...,5590.0,9950.0,8052.996670,676,103.316667,19.459241,7717.0,5590.0,9950.0,8093.375000
752065,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,1714.207071,75057.0,49.0,261000.0,34235.530303,992,47.964286,201.873995,...,47.0,78.0,63.000000,4,2.333333,12509.500000,75057.0,47.0,78.0,63.000000
752066,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,1714.207071,75057.0,49.0,261000.0,34235.530303,992,47.964286,201.873995,...,57500.0,139000.0,110844.094488,770,51.475862,6.784615,1200.0,57500.0,139000.0,111710.065646


In [44]:
features_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752068 entries, 0 to 752067
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   act_id            752068 non-null  object 
 1   iem_cd            752068 non-null  object 
 2   계좌별_평균_일매수_수량     752068 non-null  float64
 3   계좌별_최고_일매수_수량     752068 non-null  float64
 4   계좌별_매수_최소_주당금액    752068 non-null  float64
 5   계좌별_매수_최대_주당금액    752068 non-null  float64
 6   계좌별_매수_평균_주당금액    752068 non-null  float64
 7   계좌별_최대보유기간        752068 non-null  int64  
 8   계좌별_평균보유기간        752068 non-null  float64
 9   계좌별_평균_일매수매도_수량   752068 non-null  float64
 10  계좌별_최고_일매수매도_수량   752068 non-null  float64
 11  계좌별_매수매도_최소_주당금액  752068 non-null  float64
 12  계좌별_매수매도_최대_주당금액  752068 non-null  float64
 13  계좌별_매수매도_평균_주당금액  752068 non-null  float64
 14  종목별_평균_일매수_수량     752068 non-null  float64
 15  종목별_최고_일매수_수량     752068 non-null  float64
 16  종목별_매수_최소_주당금액    75

In [45]:
features_final.to_csv('홍석_feature_계좌종목O.csv', encoding='cp949', index=False)

In [46]:
features_final.iloc[:,2:].to_csv('홍석_feature.csv', encoding='cp949', index=False)